Loading necessary libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import spacy

Loading the dataset

In [2]:
df = pd.read_csv(r"recipes.csv")
df.head(5)

,id,name,url,rating,ingredients_list,num_servings,nutrition_info,ww_value,duration,course,cuisine,categories_list
0,48704,Almond Cake,https://www.skinnytaste.com/5-ingredient-almon...,4.86 from 48 votes,"eggs, egg whites, sugar, pure vanilla extract,...",12,"Serving: 1 /12th , Calories: 179 kcal ,...",6,Prep: 20 minutes mins Cook: 40 minutes ...,Dessert,American,"Baked Breads and Cakes, Dairy Free, Dessert, G..."
1,127164,Ground Turkey Taco Recipe,https://www.skinnytaste.com/ground-turkey-tacos/,5 from 5 votes,"93% lean ground turkey, taco seasoning, minced...",4,"Serving: 2 tacos , Calories: 372 kcal ,...",10,Prep: 20 minutes mins Cook: 30 minutes ...,"Dinner, main dish, Meal Prep",mexican inspired,"Cinco De Mayo, Dinner Ideas, Egg Free Recipes,..."
2,127873,Green Goddess Potato Salad,https://www.skinnytaste.com/green-goddess-pota...,NaN,"baby red potatoes, Kosher salt, packed fresh p...",6,"Serving: 1 cup , Calories: 141 kcal , ...",3,Prep: 20 minutes mins Cook: 15 minutes ...,"BBQ Side dish, Side Dish",American,"Egg Free Recipes, Gluten Free, July 4th, Lente..."
3,49576,Braised Brisket Recipe with Potatoes and Carrots,https://www.skinnytaste.com/braised-brisket-wi...,5 from 26 votes,"beef brisket, cake meal, freshly ground black ...",8,"Serving: 4 oz beef, 1`potato & carrots , C...",11,Prep: 30 minutes mins Cook: 3 hours hr...,Dinner,American,"Beef Recipes, Dairy Free, Dinner Ideas, Easter..."
4,127186,Baked Steelhead Trout Recipe,https://www.skinnytaste.com/baked-steelhead-tr...,4 from 2 votes,"package Aldi's steelhead trout fillet, extra v...",2,"Serving: 6 oz , Calories: 242 kcal , C...",1,Prep: 15 minutes mins Cook: 20 minutes ...,"Dinner, , main dish",American,"Anti Inflammatory Recipes, Dairy Free, Dinner ..."


Data Cleaning

In [3]:
# Fill NaN values with 0 before splitting
df['rating'] = df['rating'].fillna('Not rated from 0 votes')

# Split the 'rating' column and create 'rating_value' and 'rating_votes_num' columns
df[['rating_value', 'rating_votes_num']] = df['rating'].str.split(' from ', expand=True)

# Remove non-numeric characters from 'rating_votes_num' and convert to integer
df['rating_votes_num'] = df['rating_votes_num'].str.replace(' votes', '')
df['rating_votes_num'] = df['rating_votes_num'].str.replace(' vote', '')
df['rating_votes_num'] = df['rating_votes_num'].astype(int)

In [4]:
df.head()

,id,name,url,rating,ingredients_list,num_servings,nutrition_info,ww_value,duration,course,cuisine,categories_list,rating_value,rating_votes_num
0,48704,Almond Cake,https://www.skinnytaste.com/5-ingredient-almon...,4.86 from 48 votes,"eggs, egg whites, sugar, pure vanilla extract,...",12,"Serving: 1 /12th , Calories: 179 kcal ,...",6,Prep: 20 minutes mins Cook: 40 minutes ...,Dessert,American,"Baked Breads and Cakes, Dairy Free, Dessert, G...",4.86,48
1,127164,Ground Turkey Taco Recipe,https://www.skinnytaste.com/ground-turkey-tacos/,5 from 5 votes,"93% lean ground turkey, taco seasoning, minced...",4,"Serving: 2 tacos , Calories: 372 kcal ,...",10,Prep: 20 minutes mins Cook: 30 minutes ...,"Dinner, main dish, Meal Prep",mexican inspired,"Cinco De Mayo, Dinner Ideas, Egg Free Recipes,...",5,5
2,127873,Green Goddess Potato Salad,https://www.skinnytaste.com/green-goddess-pota...,Not rated from 0 votes,"baby red potatoes, Kosher salt, packed fresh p...",6,"Serving: 1 cup , Calories: 141 kcal , ...",3,Prep: 20 minutes mins Cook: 15 minutes ...,"BBQ Side dish, Side Dish",American,"Egg Free Recipes, Gluten Free, July 4th, Lente...",Not rated,0
3,49576,Braised Brisket Recipe with Potatoes and Carrots,https://www.skinnytaste.com/braised-brisket-wi...,5 from 26 votes,"beef brisket, cake meal, freshly ground black ...",8,"Serving: 4 oz beef, 1`potato & carrots , C...",11,Prep: 30 minutes mins Cook: 3 hours hr...,Dinner,American,"Beef Recipes, Dairy Free, Dinner Ideas, Easter...",5,26
4,127186,Baked Steelhead Trout Recipe,https://www.skinnytaste.com/baked-steelhead-tr...,4 from 2 votes,"package Aldi's steelhead trout fillet, extra v...",2,"Serving: 6 oz , Calories: 242 kcal , C...",1,Prep: 15 minutes mins Cook: 20 minutes ...,"Dinner, , main dish",American,"Anti Inflammatory Recipes, Dairy Free, Dinner ...",4,2


In [5]:
# Extract words after comma and before colon in nutrition_info
col = df['nutrition_info']
words = []
for index, value in col.items():
    # Check if value is NaN, replace it with an empty string
    if pd.isna(value):
        continue
    info_segments = value.split(',')
    for segment in info_segments:
        words.extend(segment.split(',')[1:])
        words.extend(segment.split(':')[:-1])

# Remove leading/trailing whitespaces and empty strings
words = [word.strip() for word in words if word.strip()]

# Get unique words
unique_words = list(set(words))

print(unique_words)


['Vitamin A', 'Fat', 'Calories', 'Cholesterol', 'Monounsaturated Fat', 'Potassium', 'Carbohydrates', 'Serving', 'Trans Fat', 'Fiber', 'Saturated Fat', 'Sodium', 'Protein', 'Iron', 'Polyunsaturated Fat', 'Sugar']


In [6]:
def parse_nutrition_info(info_str, keys):
    nutrition_dict = {key: 'Unspecified' for key in keys}

    for i, key in enumerate(keys):
        # Get the next key in the list
        next_key = keys[i + 1] if i < len(keys) - 1 else ''

        # Set pattern to capture the entire key before its value
        pattern = rf'\b{re.escape(key)}\b:\s*(.*?)(?=\b\w+\b:|$)'

        match = re.search(pattern, info_str)
        if match:
            value = match.group(1).strip()  # Trim leading and trailing spaces
            nutrition_dict[key] = value

    return nutrition_dict

# Define keys
keys = unique_words

# Apply the function to the 'nutrition_info' column and create a new column with the dictionary
df['nutrition_dict'] = df['nutrition_info'].apply(lambda x: parse_nutrition_info(str(x), keys))

print(df['nutrition_info'][3],df['nutrition_dict'][3])

Serving:  4   oz beef, 1`potato & carrots ,  Calories:  293   kcal ,  Carbohydrates:  29   g ,  Protein:  29   g ,  Fat:  6.5   g ,  Saturated Fat:  2   g ,  Cholesterol:  46.5   mg ,  Sodium:  355   mg ,  Fiber:  4.5   g ,  Sugar:  3   g {'Vitamin A': 'Unspecified', 'Fat': '6.5   g ,  Saturated', 'Calories': '293   kcal ,', 'Cholesterol': '46.5   mg ,', 'Monounsaturated Fat': 'Unspecified', 'Potassium': 'Unspecified', 'Carbohydrates': '29   g ,', 'Serving': '4   oz beef, 1`potato & carrots ,', 'Trans Fat': 'Unspecified', 'Fiber': '4.5   g ,', 'Saturated Fat': '2   g ,', 'Sodium': '355   mg ,', 'Protein': '29   g ,', 'Iron': 'Unspecified', 'Polyunsaturated Fat': 'Unspecified', 'Sugar': '3   g'}


In [7]:
# Join the keys list into a single string
keys_string = ' '.join(keys)
print(keys_string)

# Split the keys string into a list of words
word_list = keys_string.split()
print(word_list)

Vitamin A Fat Calories Cholesterol Monounsaturated Fat Potassium Carbohydrates Serving Trans Fat Fiber Saturated Fat Sodium Protein Iron Polyunsaturated Fat Sugar
['Vitamin', 'A', 'Fat', 'Calories', 'Cholesterol', 'Monounsaturated', 'Fat', 'Potassium', 'Carbohydrates', 'Serving', 'Trans', 'Fat', 'Fiber', 'Saturated', 'Fat', 'Sodium', 'Protein', 'Iron', 'Polyunsaturated', 'Fat', 'Sugar']


In [8]:
# Function to clean nutrition dictionary values
def clean_nutrition_dict(nutrition_dict):
    cleaned_nutrition_dict = {}

    # Iterate over each key-value pair in the nutrition dictionary
    for key, value in nutrition_dict.items():
        cleaned_value = value
        # Iterate over each word in the keys list
        for word in word_list:
            # Remove the word if found in the value
            cleaned_value = cleaned_value.replace(word, '')

        # Remove commas at the end of the values
        cleaned_value = cleaned_value.rstrip(',').strip()

        # Update the cleaned value in the dictionary
        cleaned_nutrition_dict[key] = cleaned_value

    return cleaned_nutrition_dict

# Apply the cleaning function to the 'nutrition_dict' column
df['nutrition_dict'] = df['nutrition_dict'].apply(clean_nutrition_dict)
print(df['nutrition_dict'][3])

{'Vitamin A': 'Unspecified', 'Fat': '6.5   g ,', 'Calories': '293   kcal', 'Cholesterol': '46.5   mg', 'Monounsaturated Fat': 'Unspecified', 'Potassium': 'Unspecified', 'Carbohydrates': '29   g', 'Serving': '4   oz beef, 1`potato & carrots', 'Trans Fat': 'Unspecified', 'Fiber': '4.5   g', 'Saturated Fat': '2   g', 'Sodium': '355   mg', 'Protein': '29   g', 'Iron': 'Unspecified', 'Polyunsaturated Fat': 'Unspecified', 'Sugar': '3   g'}


In [9]:
# Convert the 'nutrition_dict' column into separate columns
df = pd.concat([df.drop('nutrition_dict', axis=1), df['nutrition_dict'].apply(pd.Series)], axis=1)

In [10]:
# Function to extract numbers from the string
def extract_number(value):
    match = re.search(r'(\d+\.?\d*)', value)
    return match.group(1) if match else np.nan

# Apply the function to the nutrition columns
df['sugar_in_g'] = df['Sugar'].apply(lambda x: extract_number(str(x)))
df['potassium_in_mg'] = df['Potassium'].apply(lambda x: extract_number(str(x)))
df['cholesterol_in_mg'] = df['Cholesterol'].apply(lambda x: extract_number(str(x)))
df['trans_fat_in_g'] = df['Trans Fat'].apply(lambda x: extract_number(str(x)))
df['carbohydrates_in_g'] = df['Carbohydrates'].apply(lambda x: extract_number(str(x)))
df['iron_in_mg'] = df['Iron'].apply(lambda x: extract_number(str(x)))
df['vitamin_A_in_IU'] = df['Vitamin A'].apply(lambda x: extract_number(str(x)))
df['protein_in_g'] = df['Protein'].apply(lambda x: extract_number(str(x)))
df['fiber_in_g'] = df['Fiber'].apply(lambda x: extract_number(str(x)))
df['sodium_in_mg'] = df['Sodium'].apply(lambda x: extract_number(str(x)))
df['saturated_fat_in_g'] = df['Saturated Fat'].apply(lambda x: extract_number(str(x)))
df['polyunsaturated_fat_in_g'] = df['Polyunsaturated Fat'].apply(lambda x: extract_number(str(x)))
df['fat_in_g'] = df['Fat'].apply(lambda x: extract_number(str(x)))
df['monounsaturated_fat_in_g'] = df['Monounsaturated Fat'].apply(lambda x: extract_number(str(x)))
df['calories_in_kcal'] = df['Calories'].apply(lambda x: extract_number(str(x)))

In [11]:
df.head(5)

,id,name,url,rating,ingredients_list,num_servings,nutrition_info,ww_value,duration,course,...,iron_in_mg,vitamin_A_in_IU,protein_in_g,fiber_in_g,sodium_in_mg,saturated_fat_in_g,polyunsaturated_fat_in_g,fat_in_g,monounsaturated_fat_in_g,calories_in_kcal
0,48704,Almond Cake,https://www.skinnytaste.com/5-ingredient-almon...,4.86 from 48 votes,"eggs, egg whites, sugar, pure vanilla extract,...",12,"Serving: 1 /12th , Calories: 179 kcal ,...",6,Prep: 20 minutes mins Cook: 40 minutes ...,Dessert,...,NaN,NaN,7,3,39,2,NaN,11,NaN,179
1,127164,Ground Turkey Taco Recipe,https://www.skinnytaste.com/ground-turkey-tacos/,5 from 5 votes,"93% lean ground turkey, taco seasoning, minced...",4,"Serving: 2 tacos , Calories: 372 kcal ,...",10,Prep: 20 minutes mins Cook: 30 minutes ...,"Dinner, main dish, Meal Prep",...,NaN,NaN,29,5.5,NaN,6,NaN,16,NaN,372
2,127873,Green Goddess Potato Salad,https://www.skinnytaste.com/green-goddess-pota...,Not rated from 0 votes,"baby red potatoes, Kosher salt, packed fresh p...",6,"Serving: 1 cup , Calories: 141 kcal , ...",3,Prep: 20 minutes mins Cook: 15 minutes ...,"BBQ Side dish, Side Dish",...,NaN,NaN,4.5,2.5,202,0.5,NaN,3.5,NaN,141
3,49576,Braised Brisket Recipe with Potatoes and Carrots,https://www.skinnytaste.com/braised-brisket-wi...,5 from 26 votes,"beef brisket, cake meal, freshly ground black ...",8,"Serving: 4 oz beef, 1`potato & carrots , C...",11,Prep: 30 minutes mins Cook: 3 hours hr...,Dinner,...,NaN,NaN,29,4.5,355,2,NaN,6.5,NaN,293
4,127186,Baked Steelhead Trout Recipe,https://www.skinnytaste.com/baked-steelhead-tr...,4 from 2 votes,"package Aldi's steelhead trout fillet, extra v...",2,"Serving: 6 oz , Calories: 242 kcal , C...",1,Prep: 15 minutes mins Cook: 20 minutes ...,"Dinner, , main dish",...,NaN,NaN,36,3,262.5,1.5,NaN,8.5,NaN,242


In [12]:
# Define a function to clean the duration values
def clean_duration_types(duration):
    if isinstance(duration, str):
        # Define words to be removed
        words_to_remove = {"mins", "minutes", "min", "minute", "hr", "hour", "hrs", "hours", "d", "day", "days"}

        # Remove numbers and specified words
        cleaned_duration = ' '.join(word for word in duration.split() if word.lower() not in words_to_remove and not word.isdigit())

        # Split the cleaned duration by colon
        cleaned_duration_words = [word.strip() for word in cleaned_duration.split(':') if word.strip()]
        return cleaned_duration_words
    else:
        return []

# Apply the function to the 'duration' column
df['duration_types'] = df['duration'].apply(clean_duration_types)

In [13]:
df['duration_types']

0                           [Prep, Cook, Total]
1                           [Prep, Cook, Total]
2       [Prep, Cook, refrigeration time, Total]
3                           [Prep, Cook, Total]
4                           [Prep, Cook, Total]
                         ...                   
1969                        [Prep, Cook, Total]
1970                        [Prep, Cook, Total]
1971                        [Prep, Cook, Total]
1972                        [Prep, Cook, Total]
1973                                         []
Name: duration_types, Length: 1974, dtype: object

In [14]:
# Convert all values to lowercase
df['duration_types_lower'] = df['duration_types'].apply(lambda x: [word.lower() for word in x])

# Get the unique values from the 'cleaned_duration_lower' column
unique_cleaned_durations_lower = df['duration_types_lower'].explode().unique()

# Print the unique values
print(unique_cleaned_durations_lower)

['prep' 'cook' 'total' 'refrigeration time' 'cooling time' 'resting time'
 'marinade time' nan 'rest time' 'beans soak time' 'natural release time'
 'chill time' 'marinate time' 'chill and marinade time' 'soak time'
 'soak skewers' 'freeze time' 'brine time' 'soaking time' 'bean soak time'
 'sit time' 'fridge time' 'par bake crust' 'refrigerate' 'freezer time'
 'rest time for kale' 'dough resting time' 'cool time' 'chil time'
 'refrigerate time' 'release time' 'marinating time' 'marinate'
 'refrigerating time' 'crock-pot']


In [15]:
words_to_remove = {"mins", "min", "hr", "hrs", "d"}

# Construct a regex pattern to match whole words of the words to remove
pattern = r'\b(?:' + '|'.join(words_to_remove) + r')\b'

# Replace the words in the duration column using the regex pattern
df['cleaned_duration'] = df['duration'].str.replace(pattern, '', regex=True)

# Strip any extra whitespace from the duration column
df['cleaned_duration'] = df['cleaned_duration'].str.strip()

print(df['cleaned_duration'][3])

Prep:  30  minutes    Cook:  3  hours      30  minutes    Total:  4  hours


In [16]:
# Define the replacement dictionary
replace_dict = {
    r'\bprep\b': 'preparation time',
    r'\bcook\b|\bcrock-pot\b': 'cooking time',
    r'\btotal\b': 'total time',
    r'\brefrigeration time\b|\brefrigerate time\b|\brefrigerating time\b|\brefrigerate\b|\bfridge time\b': 'refrigeration time',
    r'\bchill time\b|\bcooling time\b|\bcool time\b|\bchil time\b': 'chill time',
    r'\bresting time\b|\brest time\b|\bsit time\b|\brest time for kale\b|\bdough resting time\b': 'resting time',
    r'\bmarinade time\b|\bmarinate time\b|\bmarinating time\b|\bmarinate\b|\bchill and marinade time\b': 'marinating time',
    r'\bbeans soak time\b|\bbean soak time\b|\bsoak time\b|\bsoak skewers\b|\bsoaking time\b': 'soaking time',
    r'\bnatural release time\b|\brelease time\b': 'release time',
    r'\bfreeze time\b|\bfreezer time\b': 'freezing time',
    r'\bbrine time\b': 'brining time',
    r'\bpar bake crust\b': 'partially baking crust time'
}

# Perform the replacements in the duration column
for pattern, replacement in replace_dict.items():
    df['cleaned_duration'] = df['cleaned_duration'].str.replace(pattern, replacement, regex=True, flags=re.IGNORECASE)

print(df['cleaned_duration'][3])

preparation time:  30  minutes    cooking time:  3  hours      30  minutes    total time:  4  hours


In [17]:
timekeys = ['preparation time', 'cooking time', 'total time', 'refrigeration time', 'chill time', 'resting time', 'marinating time', 'soaking time', 'release time', 'freezing time', 'brining time', 'partially baking crust time']

def extract_duration(duration_str, keys):
    duration_dict = {key: 'Not Applicable' for key in keys}
    for i, key in enumerate(keys):
        # Get the next key in the list
        next_key = keys[i + 1] if i < len(keys) - 1 else ''

        # Set pattern to capture the entire key before its value
        pattern = rf'\b{re.escape(key)}\b:\s*(.*?)(?=\b\w+\b:|$)'

        match = re.search(pattern, duration_str, re.IGNORECASE)
        if match:
            value = match.group(1).strip()  # Trim leading and trailing spaces
            value = re.sub(r'\s+', ' ', value)  # Replace consecutive spaces with a single space
            duration_dict[key] = value

    return duration_dict

# Apply the function to the 'duration' column and create a new column with the dictionary
df['duration_dict'] = df['cleaned_duration'].apply(lambda x: extract_duration(str(x), timekeys))

print(df['cleaned_duration'][2], df['duration_dict'][2])

preparation time:  20  minutes    cooking time:  15  minutes    refrigeration time:  1  hour    total time:  1  hour      35  minutes {'preparation time': '20 minutes cooking', 'cooking time': '15 minutes refrigeration', 'total time': '1 hour 35 minutes', 'refrigeration time': '1 hour total', 'chill time': 'Not Applicable', 'resting time': 'Not Applicable', 'marinating time': 'Not Applicable', 'soaking time': 'Not Applicable', 'release time': 'Not Applicable', 'freezing time': 'Not Applicable', 'brining time': 'Not Applicable', 'partially baking crust time': 'Not Applicable'}


In [18]:
# Join the keys list into a single string
timekeys_string = ' '.join(timekeys)
print(timekeys_string)

# Split the keys string into a list of words
word_list = timekeys_string.split()
print(word_list)

preparation time cooking time total time refrigeration time chill time resting time marinating time soaking time release time freezing time brining time partially baking crust time
['preparation', 'time', 'cooking', 'time', 'total', 'time', 'refrigeration', 'time', 'chill', 'time', 'resting', 'time', 'marinating', 'time', 'soaking', 'time', 'release', 'time', 'freezing', 'time', 'brining', 'time', 'partially', 'baking', 'crust', 'time']


In [19]:
# Function to clean nutrition dictionary values
def clean_duration_dict(duration_dict):
    cleaned_duration_dict = {}

    # Iterate over each key-value pair in the nutrition dictionary
    for key, value in duration_dict.items():
        cleaned_value = value
        # Iterate over each word in the keys list
        for word in word_list:
            # Remove the word if found in the value
            cleaned_value = cleaned_value.replace(word, '')

        # Update the cleaned value in the dictionary
        cleaned_duration_dict[key] = cleaned_value

    return cleaned_duration_dict

# Apply the cleaning function to the 'nutrition_dict' column
df['duration_dict'] = df['duration_dict'].apply(clean_duration_dict)
print(df['duration_dict'][2])

{'preparation time': '20 minutes ', 'cooking time': '15 minutes ', 'total time': '1 hour 35 minutes', 'refrigeration time': '1 hour ', 'chill time': 'Not Applicable', 'resting time': 'Not Applicable', 'marinating time': 'Not Applicable', 'soaking time': 'Not Applicable', 'release time': 'Not Applicable', 'freezing time': 'Not Applicable', 'brining time': 'Not Applicable', 'partially baking crust time': 'Not Applicable'}


In [20]:
# Convert the 'duration_dict' column into separate columns
df = pd.concat([df.drop('duration_dict', axis=1), df['duration_dict'].apply(pd.Series)], axis=1)

In [21]:
def convert_to_minutes(time_str):
    # Define patterns to extract time units
    patterns = {
        'days': re.compile(r'(\d+)\s*days?'),
        'hours': re.compile(r'(\d+)\s*hours?'),
        'minutes': re.compile(r'(\d+)\s*minutes?'),
    }

    total_minutes = 0

    # Extract and sum up each time unit
    if match := patterns['days'].search(time_str):
        total_minutes += int(match.group(1)) * 24 * 60
    if match := patterns['hours'].search(time_str):
        total_minutes += int(match.group(1)) * 60
    if match := patterns['minutes'].search(time_str):
        total_minutes += int(match.group(1))

    return total_minutes

# Apply the conversion function to the duration columns
df['total_time_minutes'] = df['total time'].apply(convert_to_minutes)
df['preparation_time_minutes'] = df['preparation time'].apply(convert_to_minutes)
df['cooking_time_minutes'] = df['cooking time'].apply(convert_to_minutes)
df['refrigeration_time_minutes'] = df['refrigeration time'].apply(convert_to_minutes)
df['chill_time_minutes'] = df['chill time'].apply(convert_to_minutes)
df['resting_time_minutes'] = df['resting time'].apply(convert_to_minutes)
df['marinating_time_minutes'] = df['marinating time'].apply(convert_to_minutes)
df['soaking_time_minutes'] = df['soaking time'].apply(convert_to_minutes)
df['release_time_minutes'] = df['release time'].apply(convert_to_minutes)
df['freezing_time_minutes'] = df['freezing time'].apply(convert_to_minutes)
df['brining_time_minutes'] = df['brining time'].apply(convert_to_minutes)
df['partially baking crust_time_minutes'] = df['partially baking crust time'].apply(convert_to_minutes)

In [22]:
df.head()

,id,name,url,rating,ingredients_list,num_servings,nutrition_info,ww_value,duration,course,...,cooking_time_minutes,refrigeration_time_minutes,chill_time_minutes,resting_time_minutes,marinating_time_minutes,soaking_time_minutes,release_time_minutes,freezing_time_minutes,brining_time_minutes,partially baking crust_time_minutes
0,48704,Almond Cake,https://www.skinnytaste.com/5-ingredient-almon...,4.86 from 48 votes,"eggs, egg whites, sugar, pure vanilla extract,...",12,"Serving: 1 /12th , Calories: 179 kcal ,...",6,Prep: 20 minutes mins Cook: 40 minutes ...,Dessert,...,40,0,0,0,0,0,0,0,0,0
1,127164,Ground Turkey Taco Recipe,https://www.skinnytaste.com/ground-turkey-tacos/,5 from 5 votes,"93% lean ground turkey, taco seasoning, minced...",4,"Serving: 2 tacos , Calories: 372 kcal ,...",10,Prep: 20 minutes mins Cook: 30 minutes ...,"Dinner, main dish, Meal Prep",...,30,0,0,0,0,0,0,0,0,0
2,127873,Green Goddess Potato Salad,https://www.skinnytaste.com/green-goddess-pota...,Not rated from 0 votes,"baby red potatoes, Kosher salt, packed fresh p...",6,"Serving: 1 cup , Calories: 141 kcal , ...",3,Prep: 20 minutes mins Cook: 15 minutes ...,"BBQ Side dish, Side Dish",...,15,60,0,0,0,0,0,0,0,0
3,49576,Braised Brisket Recipe with Potatoes and Carrots,https://www.skinnytaste.com/braised-brisket-wi...,5 from 26 votes,"beef brisket, cake meal, freshly ground black ...",8,"Serving: 4 oz beef, 1`potato & carrots , C...",11,Prep: 30 minutes mins Cook: 3 hours hr...,Dinner,...,210,0,0,0,0,0,0,0,0,0
4,127186,Baked Steelhead Trout Recipe,https://www.skinnytaste.com/baked-steelhead-tr...,4 from 2 votes,"package Aldi's steelhead trout fillet, extra v...",2,"Serving: 6 oz , Calories: 242 kcal , C...",1,Prep: 15 minutes mins Cook: 20 minutes ...,"Dinner, , main dish",...,20,0,0,0,0,0,0,0,0,0


In [23]:
# Preprocess the ingredients_list to remove numbers and special characters
def clean_ingredient_list(ingredient_list):
    cleaned_list = re.sub(r'[^a-zA-Z, ]', '', ingredient_list)  # Remove anything that is not a letter, comma, or space
    return cleaned_list

df['cleaned_ingredients_list'] = df['ingredients_list'].apply(clean_ingredient_list)

# Combine all ingredients into a single list
combined_ingredients = [ingredient for sublist in df['cleaned_ingredients_list'].str.split(',') for ingredient in sublist]
combined_ingredients = [ingredient.strip() for ingredient in combined_ingredients]
print(len(combined_ingredients))

unique_ingredients = list(set(filter(None, combined_ingredients)))

print(len(unique_ingredients))
print(unique_ingredients)

20951
5703
['Kosher salt and  teaspoon fresh ground pepper', 'papaya', 'crushed red pepper flakes or sriracha', 'finely grated lemon zest', 'finely chopped onion', 'lean skinless chicken drumsticks', 'highheat oil', 'chicken tenderloin', 'canned black beans', 'freezer gallon bags', 'Avocado Yogurt Dressing', 'plain yogurt', 'garlic thinly sliced', 'of saffron', 'large shallots', 'grated or shaved Parmesan cheese', 'baking spray', 'artichoke hearts', 'medium avocado', 'chocolate sprinkles', 'chopped basil or parsley', 'or dairy free parmesan', 'fine sea salt', 'uncooked brown rice', 'from  small butternut squash', 'White Nonpareils Sprinkles', 'ounces reduced fat shredded Cheddar cheese', 'Delallo Tomato Basil Pomodoro Sauce', 'chopped grape tomatoes', 'mms for the eyes', 'chipotle pepper in adobo', 'uncooked elbows', 'small onion', 'oz reduced fat crumbled feta', 'Hard Boiled Egg', 'large chicken thighs', 'large fennel bulbs', 'hot scotch bonnet pepper', 'halved yellow cherry tomatoes'

In [24]:
!pip install ingredient_parser_nlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 742.2/742.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.7/300.7 kB 21.3 MB/s eta 0:00:00


In [25]:
from ingredient_parser import parse_multiple_ingredients

# Parse ingredients
parsed_ingredients = parse_multiple_ingredients(unique_ingredients)

# Create a dictionary with sentence as key and name.text as value, removing commas in values
extracted_names_dict = {
    parsed_ingredient.sentence: parsed_ingredient.name.text.replace(',', '')
    for parsed_ingredient in parsed_ingredients if parsed_ingredient.name
}

# Print the parsed ingredients
for parsed_ingredient in parsed_ingredients:
    if parsed_ingredient.name:
        print("Name:", parsed_ingredient.name.text)
    else:
        print("Name: None")

    if parsed_ingredient.size:
        print("Size:", parsed_ingredient.size)
    else:
        print("Size: None")

    if parsed_ingredient.amount:
        print("Amount:", parsed_ingredient.amount[0].quantity, parsed_ingredient.amount[0].unit)
    else:
        print("Amount: None")

    if parsed_ingredient.preparation:
        print("Preparation:", parsed_ingredient.preparation.text)
    else:
        print("Preparation: None")

    if parsed_ingredient.comment:
        print("Comment:", parsed_ingredient.comment.text)
    else:
        print("Comment: None")

    print("Sentence:", parsed_ingredient.sentence)
    print()

# Print the extracted names dictionary
print("Extracted Names Dictionary:")
print(extracted_names_dict)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


Streaming output truncated to the last 5000 lines.
Name: None
Size: None
Amount: None
Preparation: removed from skin
Comment: None
Sentence: removed from skin

Name: grape tomatoes
Size: None
Amount: None
Preparation: None
Comment: None
Sentence: grape tomatoes

Name: full chocolate graham crackers
Size: None
Amount: None
Preparation: None
Comment: None
Sentence: full chocolate graham crackers

Name: parsley
Size: None
Amount: None
Preparation: coarsely chopped
Comment: None
Sentence: coarsely chopped parsley

Name: seasoned wheat or glutenfree panko
Size: None
Amount: None
Preparation: None
Comment: None
Sentence: seasoned wheat or glutenfree panko

Name: unsalted butter
Size: None
Amount: None
Preparation: None
Comment: None
Sentence: unsalted butter

Name: carrots
Size: None
Amount: None
Preparation: None
Comment: None
Sentence: carrots

Name: left over turkey breast
Size: None
Amount: None
Preparation: None
Comment: None
Sentence: left over turkey breast

Name: seasoned whole wheat

In [26]:
# List of words to remove
words_to_remove = ['cup', 'cups', 'about', 'oz', 'teaspoons', 'firmly', 'optional', 'more', 'packed', 'finely', 'chopped', 'reduced', 'diced', 'grated', 'ground', 'minced', 'shredded', 'sliced', 'peeled', 'large', 'bunch', 'mild', 'fresh', 'each', 'total', 'unpacked', 'unpeeled', 'petite', 'small', 'tbsp', 'dash', 'freshly', 'taste', 'dashes', 'few', 'little', 'slices', 'thin', 'thinly', 'lb', 'very', 'can', 'cans', 'canned', 'ounce', 'ounces', 'medium', 'prepared', 'sprig', 'sprigs', 'mixed', 'frozen', 'pieces', 'drops', 'plus', 'to', 'a', 'ultra', 'round', 'rounds', 'package', 'packaged', 'piece', 'inch', 'inches', 'drop', 'drops', 'pinch', 'pinches', 'trimmed', 'cubed', 'cubes', 'strips', 'strip', 'cut', 'into', 'center', 'dry', 'centercut', 'less', 'tsp', 'th', 'leftover', 'lengthwise', 'lenghtwise', 'loosely', 'loose', 'half', 'halved', 'quartered', 'quarters', 'chunks', 'bite', 'sized', 'bitesized', 'bitesize', 'even', 'sprinkle', 'head', 'drizzle', 'scoop', 'add', 'handful', 'generous', 'Drizzle', 'Handful', 'aof', 'scoops', 'pound', 'pounds', 'cleaned', 'cooked', 'extra', 'uncooked', 'fully', 'bonein', 'boneless', 'skinned', 'skinless', 'skinon', 'crushed', 'crumbled', 'dice', 'drained', 'rinsed', 'well', 'teaspoon', 'tablespoons', 'squeezed', 'tablespoon', 'squirt', 'squirts', 'squeeze', 'slice', 'skin', 'removed', 'shaved', 'sharp', 'seedless', 'seeded', 'roughly', 'refrigerated', 'pitted', 'husked', 'split']
words_to_remove_beginning_end = ['of', 'and', 'or', 'from']

# Function to replace list values with dictionary values if keys match
def replace_with_parsed(ingredient_list):
    parsed_list = [
        extracted_names_dict.get(ingredient, ingredient)
        for ingredient in ingredient_list
    ]
    return parsed_list

# Function to remove specific words from the list values
def replace_with_cleaned(ingredient_list):
    cleaned_list = []
    for ingredient in ingredient_list:
        # Split the ingredient into words
        words = ingredient.split()

        # Remove the specified words
        filtered_words = [word for word in words if word.lower() not in words_to_remove]

        # Remove words from the beginning or end
        while filtered_words and filtered_words[0].lower() in words_to_remove_beginning_end:
            filtered_words.pop(0)
        while filtered_words and filtered_words[-1].lower() in words_to_remove_beginning_end:
            filtered_words.pop()

        # Join the filtered words back into a string
        cleaned_list.append(' '.join(filtered_words))

    return cleaned_list

# Function to replace ingredients that start with 'for' with a blank string
def remove_for_ingredients(ingredient_list):
    return ['' if ingredient.strip().lower().startswith('for') else ingredient for ingredient in ingredient_list]

# Apply the replacement to each row
df['parsed_ingredients_list'] = df['cleaned_ingredients_list'].apply(lambda x: replace_with_parsed(x.split(',')))

# Apply the word removal to each row
df['final_ingredients_list'] = df['parsed_ingredients_list'].apply(replace_with_cleaned)

# Apply the 'for' removal to each row
df['final_ingredients_list'] = df['final_ingredients_list'].apply(remove_for_ingredients)

# Print the first row to check the result
print(df['parsed_ingredients_list'][1306])
print(df['final_ingredients_list'][1306])

['cooking spray', ' olive oil', ' garlic cloves', ' unpeeled zucchini', ' kosher salt', ' black pepper', ' chopped basil', ' Italian seasoned breadcrumbs', ' large egg', '  cup Pecorino Romano cheese', ' freshly grated', ' plus more for serving', ' quick marinara sauce', '  or jarred', ' part skim ricotta cheese']
['cooking spray', 'olive oil', 'garlic cloves', 'zucchini', 'kosher salt', 'black pepper', 'basil', 'Italian seasoned breadcrumbs', 'egg', 'Pecorino Romano cheese', '', '', 'quick marinara sauce', 'jarred', 'part skim ricotta cheese']


In [27]:
# Define a function to clean the course values
def clean_course(course):
    if isinstance(course, str):
        # Define the regular expression pattern
        pattern = r',+\s*,+'
        # Replace consecutive commas followed by whitespace with a single comma
        cleaned_course = course
        while re.search(pattern, cleaned_course):
            cleaned_course = re.sub(pattern, ',', cleaned_course)
        # Strip leading and trailing commas
        cleaned_course = cleaned_course.strip(',')
        # Split cleaned course string into a list
        cleaned_course_list = [item.strip() for item in cleaned_course.split(',') if item.strip()]
        return cleaned_course_list
    else:
        return []

# Apply the function to the 'course' column
df['course_cleaned'] = df['course'].apply(clean_course)

df.head(10)

,id,name,url,rating,ingredients_list,num_servings,nutrition_info,ww_value,duration,course,...,marinating_time_minutes,soaking_time_minutes,release_time_minutes,freezing_time_minutes,brining_time_minutes,partially baking crust_time_minutes,cleaned_ingredients_list,parsed_ingredients_list,final_ingredients_list,course_cleaned
0,48704,Almond Cake,https://www.skinnytaste.com/5-ingredient-almon...,4.86 from 48 votes,"eggs, egg whites, sugar, pure vanilla extract,...",12,"Serving: 1 /12th , Calories: 179 kcal ,...",6,Prep: 20 minutes mins Cook: 40 minutes ...,Dessert,...,0,0,0,0,0,0,"eggs, egg whites, sugar, pure vanilla extract,...","[eggs, egg whites, sugar, pure vanilla extr...","[eggs, egg whites, sugar, pure vanilla extract...",[Dessert]
1,127164,Ground Turkey Taco Recipe,https://www.skinnytaste.com/ground-turkey-tacos/,5 from 5 votes,"93% lean ground turkey, taco seasoning, minced...",4,"Serving: 2 tacos , Calories: 372 kcal ,...",10,Prep: 20 minutes mins Cook: 30 minutes ...,"Dinner, main dish, Meal Prep",...,0,0,0,0,0,0,"lean ground turkey, taco seasoning, minced ga...","[ lean ground turkey, taco seasoning, minced...","[lean turkey, taco seasoning, garlic, onion, r...","[Dinner, main dish, Meal Prep]"
2,127873,Green Goddess Potato Salad,https://www.skinnytaste.com/green-goddess-pota...,Not rated from 0 votes,"baby red potatoes, Kosher salt, packed fresh p...",6,"Serving: 1 cup , Calories: 141 kcal , ...",3,Prep: 20 minutes mins Cook: 15 minutes ...,"BBQ Side dish, Side Dish",...,0,0,0,0,0,0,"baby red potatoes, Kosher salt, packed fresh p...","[baby red potatoes, Kosher salt, packed fres...","[baby red potatoes, Kosher salt, parsley leave...","[BBQ Side dish, Side Dish]"
3,49576,Braised Brisket Recipe with Potatoes and Carrots,https://www.skinnytaste.com/braised-brisket-wi...,5 from 26 votes,"beef brisket, cake meal, freshly ground black ...",8,"Serving: 4 oz beef, 1`potato & carrots , C...",11,Prep: 30 minutes mins Cook: 3 hours hr...,Dinner,...,0,0,0,0,0,0,"beef brisket, cake meal, freshly ground black ...","[beef brisket, cake meal, freshly ground bla...","[beef brisket, cake meal, black pepper, olive ...",[Dinner]
4,127186,Baked Steelhead Trout Recipe,https://www.skinnytaste.com/baked-steelhead-tr...,4 from 2 votes,"package Aldi's steelhead trout fillet, extra v...",2,"Serving: 6 oz , Calories: 242 kcal , C...",1,Prep: 15 minutes mins Cook: 20 minutes ...,"Dinner, , main dish",...,0,0,0,0,0,0,"package Aldis steelhead trout fillet, extra vi...","[Aldis steelhead trout fillet, extra virgin o...","[Aldis steelhead trout fillet, virgin olive oi...","[Dinner, main dish]"
5,127355,Italian Shrimp Salad,https://www.skinnytaste.com/italian-shrimp-salad/,4.84 from 6 votes,"cooked, sliced celery, good quality mixed Ital...",6,"Serving: 1 generous cup , Calories: 198 ...",4,Prep: 20 minutes mins Cook: 0 minutes ...,"Appetizer, , Lunch, Meal Prep, Salad",...,0,0,0,0,0,0,"cooked, sliced celery, good quality mixed Ital...","[cooked, sliced celery, good quality mixed I...","[, celery, good quality Italian olives, red on...","[Appetizer, Lunch, Meal Prep, Salad]"
6,47636,Cream of Asparagus Soup Recipe,https://www.skinnytaste.com/cream-of-asparagus...,4.83 from 102 votes,"asparagus, unsalted butter, medium onion, redu...",6,"Serving: 1 1/4 cups , Calories: 81 kcal...",1,Prep: 5 minutes mins Cook: 20 minutes ...,Soup,...,0,0,0,0,0,0,"asparagus, unsalted butter, medium onion, redu...","[asparagus, unsalted butter, medium onion, ...","[asparagus, unsalted butter, onion, sodium chi...",[Soup]
7,48094,Grilled Salmon Bruschetta with Avocado,https://www.skinnytaste.com/grilled-salmon-wit...,4.92 from 25 votes,"wild salmon fillets, kosher salt, black pepper...",4,"Serving: 1 salmon filet , Calories: 340.5...",3,Prep: 10 minutes mins Cook: 10 minutes ...,Dinner,...,0,0,0,0,0,0,"wild salmon fillets, kosher salt, black pepper...","[wild salmon fillets, kosher salt, black pep...","[wild salmon fillets, kosher salt, black peppe...",[Dinner]
8,48686,Tu

In [28]:
# Split the strings into lists
df['cuisine_list'] = df['cuisine'].apply(lambda x: [c.strip() for c in x.split(',')])

In [29]:
# Function to split strings into lists, handling NaN values
def split_and_clean(categories_str):
    if isinstance(categories_str, str):
        return [c.strip() for c in categories_str.split(',')]
    else:
        return []

# Apply the function to create a list of lists
df['categories_list'] = df['categories_list'].apply(split_and_clean)

Data Modeling

In [30]:
# Flatten the lists from each row into a single list
all_ingredients = [ingredient for sublist in df['final_ingredients_list'] if sublist for ingredient in sublist if ingredient]

# Convert the flattened list to a set to get unique values
unique_names = list(set(all_ingredients))

# Create a DataFrame with 'id' and 'ingredient name' columns
ingredient_df = pd.DataFrame({'ingredient_id': range(1, len(unique_names)+1), 'ingredient_name': unique_names})

# Save the DataFrame to a CSV file
ingredient_df.to_csv('ingredient_list.csv', index=False)

In [31]:
# Flatten the lists from each row into a single list
all_courses = [course for sublist in df['course_cleaned'] for course in sublist]

# Convert the flattened list to a set to get unique values
unique_courses = list(set(all_courses))

# Create a DataFrame with 'id' and 'course name' columns
course_df = pd.DataFrame({'course_id': range(1, len(unique_courses)+1), 'course_name': unique_courses})

# Save the DataFrame to a CSV file
course_df.to_csv('course_list.csv', index=False)

In [32]:
# Flatten the lists from each row into a single list
all_cuisines = [cuisine.strip() for sublist in df['cuisine_list'] for cuisine in sublist]

# Convert the flattened list to a set to get unique values
unique_cuisines = list(set(all_cuisines))

# Create a DataFrame with 'id' and 'cuisine name' columns
cuisine_df = pd.DataFrame({'cuisine_id': range(1, len(unique_cuisines)+1), 'cuisine_name': unique_cuisines})

# Save the DataFrame to a CSV file
cuisine_df.to_csv('cuisine_list.csv', index=False)

In [33]:
# Flatten the list of lists into a single list
all_categories = [category.strip() for sublist in df['categories_list'] for category in sublist]

# Get unique categories by converting to a set and back to a list
unique_categories = list(set(all_categories))

# Create a DataFrame with 'id' and 'category_name' columns
categories_df = pd.DataFrame({'category_id': range(1, len(unique_categories) + 1), 'category_name': unique_categories})

# Save the DataFrame to a CSV file
categories_df.to_csv('category_list.csv', index=False)

Relational Mapping

In [34]:
# Explode the ingredient column
df_explodedingredient = df.explode('final_ingredients_list')

# Merge with the ingredient dataframe to get ingredient IDs
df_finalingredient = df_explodedingredient.merge(ingredient_df, left_on='final_ingredients_list', right_on='ingredient_name', how='left')

# Ensure ingredient_id is integer
df_finalingredient['ingredient_id'] = df_finalingredient['ingredient_id'].fillna(-1).astype(int)

# Select the required columns
recipe_ingredient_df = df_finalingredient[['id', 'ingredient_id']]
recipe_ingredient_df.rename(columns={'id': 'recipe_id'}, inplace=True)
print(recipe_ingredient_df)

# Save the DataFrame to a CSV file
recipe_ingredient_df.to_csv('recipe_ingredient.csv', index=False)

       recipe_id  ingredient_id
0          48704            599
1          48704           2991
2          48704           1894
3          48704           2946
4          48704             60
...          ...            ...
20946      50500            393
20947      50500           2043
20948      50500           2138
20949      50500            886
20950      50500           3379

[20951 rows x 2 columns]


<ipython-input-34-e05fcaa584ee>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_ingredient_df.rename(columns={'id': 'recipe_id'}, inplace=True)


In [35]:
# Explode the course column
df_explodedcourse = df.explode('course_cleaned')

# Merge with the course dataframe to get course IDs
df_finalcourse = df_explodedcourse.merge(course_df, left_on='course_cleaned', right_on='course_name', how='left')
# Ensure course_id is integer
df_finalcourse['course_id'] = df_finalcourse['course_id'].fillna(-1).astype(int)

# Select the required columns
recipe_course_df = df_finalcourse[['id', 'course_id']]
recipe_course_df.rename(columns={'id': 'recipe_id'}, inplace=True)
print(recipe_course_df)

# Save the DataFrame to a CSV file
recipe_course_df.to_csv('recipe_course.csv', index=False)

      recipe_id  course_id
0         48704         20
1        127164          6
2        127164         26
3        127164         23
4        127873         24
...         ...        ...
3252      50504         21
3253      50506          6
3254      50500          2
3255      50500         18
3256      50500          9

[3257 rows x 2 columns]


<ipython-input-35-64db95fa4c43>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_course_df.rename(columns={'id': 'recipe_id'}, inplace=True)


In [36]:
# Explode the cuisine column
df_explodedcuisine = df.explode('cuisine_list')

# Merge with the cuisine dataframe to get cuisine IDs
df_finalcuisine = df_explodedcuisine.merge(cuisine_df, left_on='cuisine_list', right_on='cuisine_name', how='left')
# Ensure cuisine_id is integer
df_finalcuisine['cuisine_id'] = df_finalcuisine['cuisine_id'].fillna(-1).astype(int)

# Select the required columns
recipe_cuisine_df = df_finalcuisine[['id', 'cuisine_id']]
recipe_cuisine_df.rename(columns={'id': 'recipe_id'}, inplace=True)
print(recipe_cuisine_df)

# Save the DataFrame to a CSV file
recipe_cuisine_df.to_csv('recipe_cuisine.csv', index=False)

      recipe_id  cuisine_id
0         48704           2
1        127164          30
2        127873           2
3         49576           2
4        127186           2
...         ...         ...
2164      50502           2
2165      50488           2
2166      50504           2
2167      50506          54
2168      50500           2

[2169 rows x 2 columns]


<ipython-input-36-e834d8223fc4>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_cuisine_df.rename(columns={'id': 'recipe_id'}, inplace=True)


In [37]:
# Explode the categories column
df_explodedcategories = df.explode('categories_list')

# Merge with the categories dataframe to get categories IDs
df_finalcategories = df_explodedcategories.merge(categories_df, left_on='categories_list', right_on='category_name', how='left')
# Ensure categories_id is integer
df_finalcategories['category_id'] = df_finalcategories['category_id'].fillna(-1).astype(int)

# Select the required columns
recipe_categories_df = df_finalcategories[['id', 'category_id']]
recipe_categories_df.rename(columns={'id': 'recipe_id'}, inplace=True)
print(recipe_categories_df)

# Save the DataFrame to a CSV file
recipe_categories_df.to_csv('recipe_category.csv', index=False)

       recipe_id  category_id
0          48704           33
1          48704          104
2          48704           52
3          48704          108
4          48704           87
...          ...          ...
23825      50500          109
23826      50500           75
23827      50500           23
23828      50500           29
23829      50500           65

[23830 rows x 2 columns]


<ipython-input-37-d7865f80d72c>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipe_categories_df.rename(columns={'id': 'recipe_id'}, inplace=True)


In [38]:
df.head(5)

,id,name,url,rating,ingredients_list,num_servings,nutrition_info,ww_value,duration,course,...,soaking_time_minutes,release_time_minutes,freezing_time_minutes,brining_time_minutes,partially baking crust_time_minutes,cleaned_ingredients_list,parsed_ingredients_list,final_ingredients_list,course_cleaned,cuisine_list
0,48704,Almond Cake,https://www.skinnytaste.com/5-ingredient-almon...,4.86 from 48 votes,"eggs, egg whites, sugar, pure vanilla extract,...",12,"Serving: 1 /12th , Calories: 179 kcal ,...",6,Prep: 20 minutes mins Cook: 40 minutes ...,Dessert,...,0,0,0,0,0,"eggs, egg whites, sugar, pure vanilla extract,...","[eggs, egg whites, sugar, pure vanilla extr...","[eggs, egg whites, sugar, pure vanilla extract...",[Dessert],[American]
1,127164,Ground Turkey Taco Recipe,https://www.skinnytaste.com/ground-turkey-tacos/,5 from 5 votes,"93% lean ground turkey, taco seasoning, minced...",4,"Serving: 2 tacos , Calories: 372 kcal ,...",10,Prep: 20 minutes mins Cook: 30 minutes ...,"Dinner, main dish, Meal Prep",...,0,0,0,0,0,"lean ground turkey, taco seasoning, minced ga...","[ lean ground turkey, taco seasoning, minced...","[lean turkey, taco seasoning, garlic, onion, r...","[Dinner, main dish, Meal Prep]",[mexican inspired]
2,127873,Green Goddess Potato Salad,https://www.skinnytaste.com/green-goddess-pota...,Not rated from 0 votes,"baby red potatoes, Kosher salt, packed fresh p...",6,"Serving: 1 cup , Calories: 141 kcal , ...",3,Prep: 20 minutes mins Cook: 15 minutes ...,"BBQ Side dish, Side Dish",...,0,0,0,0,0,"baby red potatoes, Kosher salt, packed fresh p...","[baby red potatoes, Kosher salt, packed fres...","[baby red potatoes, Kosher salt, parsley leave...","[BBQ Side dish, Side Dish]",[American]
3,49576,Braised Brisket Recipe with Potatoes and Carrots,https://www.skinnytaste.com/braised-brisket-wi...,5 from 26 votes,"beef brisket, cake meal, freshly ground black ...",8,"Serving: 4 oz beef, 1`potato & carrots , C...",11,Prep: 30 minutes mins Cook: 3 hours hr...,Dinner,...,0,0,0,0,0,"beef brisket, cake meal, freshly ground black ...","[beef brisket, cake meal, freshly ground bla...","[beef brisket, cake meal, black pepper, olive ...",[Dinner],[American]
4,127186,Baked Steelhead Trout Recipe,https://www.skinnytaste.com/baked-steelhead-tr...,4 from 2 votes,"package Aldi's steelhead trout fillet, extra v...",2,"Serving: 6 oz , Calories: 242 kcal , C...",1,Prep: 15 minutes mins Cook: 20 minutes ...,"Dinner, , main dish",...,0,0,0,0,0,"package Aldis steelhead trout fillet, extra vi...","[Aldis steelhead trout fillet, extra virgin o...","[Aldis steelhead trout fillet, virgin olive oi...","[Dinner, main dish]",[American]


In [40]:
df = df.drop(columns=['url', 'rating', 'ingredients_list', 'cleaned_ingredients_list', 'parsed_ingredients_list', 'nutrition_info', 'Sugar', 'Potassium', 'Cholesterol', 'Trans Fat', 'Carbohydrates', 'Iron', 'Vitamin A', 'Protein', 'Fiber', 'Sodium', 'Saturated Fat', 'Calories', 'Polyunsaturated Fat', 'Fat', 'Monounsaturated Fat', 'duration', 'duration_types', 'duration_types_lower', 'cleaned_duration', 'course', 'cuisine', 'preparation time', 'cooking time', 'total time', 'refrigeration time', 'chill time', 'resting time', 'marinating time', 'soaking time', 'release time', 'freezing time', 'brining time', 'partially baking crust time'])


In [41]:
# Specify the file path where you want to save the CSV file
file_path = "recipes_cleaned.csv"

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)